In [ ]:
# 🚀 PRODUCTION BATCH INFERENCE 
 
from databricks.sdk import WorkspaceClient
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import yaml
import traceback
import sys  # ← ADDED
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

print("=" * 80)
print("🚀 PRODUCTION INFERENCE PIPELINE (NEW WORKFLOW)")
print("=" * 80)
 
# 1️⃣ Load pipeline configuration
 
print("\n📋 Loading pipeline_config.yml...")

import os

try:
    config_path = "/Workspace/Repos/vipultak7171@gmail.com/ml-credit-risk/dev_env/pipeline_config.yml"
    
    if not os.path.exists(config_path):
        config_path = "pipeline_config.yml"  # fallback if repo path fails

    with open(config_path, "r") as f:
        pipeline_cfg = yaml.safe_load(f)

    MODEL_TYPE = pipeline_cfg["model"]["type"]
    BASE_NAME = pipeline_cfg["model"]["base_name"]

    # ✅ NEW: Dynamic endpoint name from YAML format rule
    ENDPOINT_NAME = pipeline_cfg["serving"]["endpoint_name_format"].format(
        base_name=BASE_NAME.replace("_", "-"),
        model_type=MODEL_TYPE
    )

    data_cfg = pipeline_cfg["data"]
    INPUT_TABLE = data_cfg["input_table"]
    FEATURES = data_cfg["features"]
    LABEL = data_cfg["label"]

    # ✅ NEW: Dynamic output table based on YAML format rule
    inference_cfg = pipeline_cfg.get("inference", {})
    OUTPUT_TABLE = inference_cfg["output_table_format"].format(
        catalog=pipeline_cfg["model"]["catalog"],
        schema=pipeline_cfg["model"]["schema"],
        model_type=MODEL_TYPE
    )

    BATCH_SIZE = inference_cfg.get("batch_size", 100)

    print(f"📌 Using configs:")
    print(f"   ➤ Endpoint: {ENDPOINT_NAME}")
    print(f"   ➤ Input Table: {INPUT_TABLE}")
    print(f"   ➤ Output Table: {OUTPUT_TABLE}")
    print(f"   ➤ Batch Size: {BATCH_SIZE}")

except Exception as e:
    print(f"❌ Failed to load config: {e}")
    traceback.print_exc()
    sys.exit(1)
 
# 2️⃣ Initialize clients

try:
    spark = SparkSession.builder.appName("ProductionInference").getOrCreate()
    ws = WorkspaceClient()
    print("\n✅ Spark + WorkspaceClient initialized")

except Exception as e:
    print(f"❌ Initialization failed: {e}")
    traceback.print_exc()
    sys.exit(1)
 
# 3️⃣ Load input data
 
print(f"\n📥 Loading data from {INPUT_TABLE}...")

try:
    df = spark.read.table(INPUT_TABLE).toPandas()

    # Validate required features
    missing = [col for col in FEATURES if col not in df.columns]
    if missing:
        raise ValueError(f"Missing input features: {missing}")

    if LABEL not in df.columns:
        print(f"⚠️ No label column found → metrics will be skipped")

    print(f"📌 Total records: {len(df):,}")

except Exception as e:
    print(f"❌ Data load error: {e}")
    traceback.print_exc()
    sys.exit(1)
 
# 4️⃣ Run inference using Serving Endpoint

print("\n🔄 Running inference through serving endpoint...")

predictions = []
n_batches = (len(df) // BATCH_SIZE) + 1

try:
    for i in range(n_batches):
        batch = df.iloc[i * BATCH_SIZE : (i+1) * BATCH_SIZE]

        if batch.empty:
            continue

        response = ws.serving_endpoints.query(
            name=ENDPOINT_NAME,
            dataframe_records=batch[FEATURES].to_dict("records")
        )

        predictions.extend(response.predictions)
        
        if (i + 1) % 10 == 0 or (i + 1) == n_batches:
            print(f"   Processed batch {i+1}/{n_batches}")

    print(f"✅ Inference complete: {len(predictions)} predictions generated")

except Exception as e:
    print(f"❌ Inference failed: {e}")
    traceback.print_exc()
    sys.exit(1)
 
# 5️⃣ Build output dataframe
 
print("\n📦 Preparing output results...")

df["prediction"] = predictions

# 🔧 Ensure prediction is integer before converting
df["prediction"] = pd.to_numeric(df["prediction"], errors="coerce").astype("Int64")

# 🔥 Convert prediction to human readable Yes/No
label_mapping = {0: "No", 1: "Yes"}
df["prediction_label"] = df["prediction"].map(label_mapping)


df["prediction_timestamp"] = datetime.now()
df["model_type"] = MODEL_TYPE.upper()
df["endpoint"] = ENDPOINT_NAME
 
# 6️⃣ Save to Delta Table
 
print(f"\n💾 Saving results to {OUTPUT_TABLE}...")

try:
    spark_df = spark.createDataFrame(df)

    spark_df.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable(OUTPUT_TABLE)

    print(f"✅ Results saved to: {OUTPUT_TABLE}")

except Exception as e:
    print(f"⚠️ Write failure but continuing: {e}")
    traceback.print_exc()
 
# 7️⃣ Compute model performance (if true labels present)

if LABEL in df.columns:
    print("\n📊 Evaluating model performance (Classification)...")

    # 🔧 FIX seed true labels to match 0/1 type
    y_true = df[LABEL].replace({"no": 0, "yes": 1}).astype(int)
    y_pred = df["prediction"].astype(int)

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

    try:
        metrics["roc_auc"] = roc_auc_score(y_true, y_pred)
    except:
        metrics["roc_auc"] = None

    print("📊 Metrics:")
    for k, v in metrics.items():
        if v is not None:
            print(f"   ➤ {k}: {round(v,4)}")

else:
    print("\nℹ️ No true labels found → skipping metrics.")
 
# 8️⃣ Final Summary

print("\n" + "="*80)
print(f"🎯 PRODUCTION INFERENCE COMPLETED")
print("="*80)
print(f"📌 Input Data     : {INPUT_TABLE}")
print(f"📌 Saved Results  : {OUTPUT_TABLE}")
print(f"📌 Predictions    : {len(df):,}")
print(f"📌 Endpoint       : {ENDPOINT_NAME}")
print("="*80)

print("✅ Script completed successfully")
